In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob
import scipy.stats
from IPython.display import Video, display, HTML
sns.set()

def read_csvx(path, *args, **kwargs):
    try:
        return pd.read_csv(str(path) +'.gz', *args, **kwargs)
    except Exception as e:
        return pd.read_csv(str(path), *args, **kwargs)

def followorbit(vals, path):
    out = []
    for n in range(100):
        closest05 = np.clip(round(path[n] * 2) / 2, 0, 10)
        rank = vals.query(f'nbopp == {n} and othercoop == {closest05}')['rank'].values[0]
        out.append(rank)
    return np.array(out)

import re
def gethighest(vals, prefix="logall_", which=1):
    if which == 1:
        m = np.argmax([int(re.search(f'{prefix}(\d+)', val).group(1)) for val in vals])
    else:
        trueval = [int(re.search(f'{prefix}(\d+)', val).group(1)) for val in vals]
        itrueval = sorted(list(enumerate(trueval)), key=lambda x: x[1])
        m = itrueval[-which][0]
    return vals[m]

    
def readparams(file):
    params = {} 
    with open(file) as f:
        for line in f:
            line = line[:-1]  # remove \n
            if not line or line.startswith('#') or line.startswith('import'):
                continue
            key, val = line.split('=')
            if val.lower() == 'true' or val.lower() == 'false':
                params[key] = val.lower() == 'true'
            else:
                try:
                    params[key] = float(val)
                except ValueError:
                    params[key] = val
    return params

def sorted_nicely( l ): 
    """ Sort the given iterable in the way that humans expect.""" 
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

def readparams(file):
    params = {} 
    with open(file) as f:
        for line in f:
            line = line[:-1]  # remove \n
            if not line or line.startswith('#') or line.startswith('import'):
                continue
            key, val = line.split('=')
            if val.lower() == 'true' or val.lower() == 'false':
                params[key] = val.lower() == 'true'
            else:
                try:
                    params[key] = float(val)
                except ValueError:
                    params[key] = val
    return params

import platform
hostname = platform.node()
print(hostname)

if hostname == 'cluster':
    prefix = '/home/ecoffet/robocoop/'
else:
    prefix = '/home/pecoffet/remoterobo/'

In [ ]:
import json
paths = sorted(glob.glob(f'{prefix}/logs/negociate2/*faketrue2*/**/rep00/', recursive=True))
paths = ['/home/pecoffet/remoterobo//logs/negociate2/negociate-faketrue2-mutCoop0.01-uni0.001-2019-08-19-1539/run_14/rep00/', # 13
'/home/pecoffet/remoterobo//logs/negociate2/negociate-faketrue2-mutCoop0.01-uni0.001-2019-08-19-1539/run_23/rep00/', # 22
'/home/pecoffet/remoterobo//logs/negociate2/negociate-fit-faketrue2-mutCoop0.01-uni0.001-2019-08-19-1540/run_23/rep00/', # 46
'/home/pecoffet/remoterobo//logs/negociate2/negociate-fit-faketrue2-mutCoop0.01-uni0.001-2019-08-19-1540/run_06/rep00/', # 29
]
meancoop = []

for path in paths:
    if 'train' in path:
        continue
    try:
        print(path)
        plt.figure()
        fitness = read_csvx(path+'../fit.txt', delimiter="\t", names=['gen', 'min', 'q1', 'med', 'q3', 'max'])
        fitness['med'].plot()
        plt.title('Fitness médiane')
        plt.show()
        try:
            plt.figure()
            coop = read_csvx(gethighest(glob.glob(path + '/coop*'), 'coop_'), delimiter="\t")
            sns.distplot(coop['coop'], kde=False, bins=np.arange(0, 10.1, 0.5))
            plt.xlim(0, 10)
            plt.title('Distribution de l\'investissement à la dernière génération')
            plt.show()
            meancoop.append(coop['coop'].mean())
        except Exception as e:
            print(e)
        # genome graph
        allgenomefiles = sorted_nicely(glob.glob(path + 'genomes_*.txt'))
        params = readparams(glob.glob(path + 'properties*')[0])
        out = []
        for genomefile in allgenomefiles:
            generation = int(re.search('genomes_([0-9]+).txt', genomefile).group(1))
            with open(genomefile) as gf:
                gen = np.asarray(json.load(gf))
                coopgen = gen[:, 0]
                coopval = coopgen * params['maxCoop']
            for i, val in enumerate(coopval):
                out.append({'gen': generation, 'ind': i, 'coop': val})
        coopdf = pd.DataFrame(out)
        try:
            fig = plt.figure()
            ax = fig.gca()
            ax.set_ylim(0, params['maxCoop'])
            sns.lineplot(x='gen', y='coop', data=coopdf, ax=ax)
            plt.title('Evolution de l\'investissement moyen par génération')
            plt.show(fig)
        except Exception as e:
            print(e)
        if False:
            try:
                choice = read_csvx(gethighest(glob.glob(path + '/analysis*'), 'analysis_log_', 2), delimiter="\t")
                choice['other'] = np.round(choice['other'], 2)
                choice['owncoop'] = np.round(choice['owncoop'], 2)
                sns.heatmap(pd.pivot_table(choice, 'accept', 'owncoop', 'other'))
                plt.show()
            except Exception as e:
                print(e)
        fig = plt.figure()
        ax = fig.gca()
        choice = read_csvx(gethighest(glob.glob(path + '/analysis*'), 'analysis_log_'), delimiter="\t")
        choice['other'] = np.round(choice['other'], 1)
        choice['owncoop'] = np.round(choice['owncoop'], 1)
        sns.heatmap(pd.pivot_table(choice, 'accept', 'other', 'owncoop'), cmap=sns.cm.rocket_r, vmin=0, vmax=1, ax=ax)
        ax.invert_yaxis()
        plt.title('Pourcentage des agents qui acceptent un jeu à la dernière génération')
        plt.show()
        print('end')

    except Exception as e:
        print(e)

In [ ]:
meancoops = np.asarray(meancoop)
sns.distplot(meancoops)

In [ ]:
nESS = len(meancoops[(2 < meancoops) & (meancoops < 3)])
nSO = len(meancoops[(5 < meancoops) & (meancoops < 7)])
nOther = len(meancoops - nESS - nSO)
sorted(range(len(meancoops)), key=lambda k: meancoops[k], reverse=True)

In [ ]:
meancoops

In [ ]:
binned = pd.DataFrame({'coop': meancoops, 'bin':pd.cut(meancoops, range(0, 11, 1))})

binned.groupby('bin').count()

In [ ]:
import tqdm
paths = sorted(glob.glob(f'{prefix}/logs/negociate2/*faketrue2*/**/rep00/', recursive=True))

binstep = 1

out = []

for irun, path in enumerate(tqdm.tqdm_notebook(paths)):
    if 'train' in path:
        continue
#    print(path)
    
    coop = read_csvx(gethighest(glob.glob(path + '/coop*'), 'coop_'), delimiter="\t")

    meancoop = coop['coop'].mean()
    
    df = read_csvx(path + '/replay/logall_0.txt', delimiter="\t")
    df['realcoop1'] = df['trueCoop1'] + df['fakeCoef1']
    df['realcoop2'] = df['trueCoop2'] + df['fakeCoef2']
    df['Accept1'] = df['Accept1'].astype(bool)
    df['Accept2'] = df['Accept2'].astype(bool)
    df['bin1'] = pd.cut(df['realcoop1'], range(0, 11, binstep))
    df['bin2'] = pd.cut(df['realcoop2'], range(0, 11, binstep))
    df['bin'] = '_other'
    df.loc[df['bin1'] == df['bin2'], 'bin'] = df['bin1']
    df['bin'] = df['bin'].astype(str)
    df['bothaccept'] = df['Accept1'] & df['Accept2']
    toadd = {'id': irun,
                 'meanCoop': meancoop,
                'path': path}
    for bin_ in df['bin'].unique():
        toadd[bin_ + '_accept'] = df.query(f'bin == "{bin_}" and bothaccept == True').count()['iter']
        toadd[bin_ + '_reject'] = df.query(f'bin == "{bin_}" and bothaccept == False').count()['iter']
    out.append(toadd)
                    

In [ ]:
acceptation = pd.DataFrame(out).fillna(0)


In [ ]:
sacceptation = acceptation.copy()
del sacceptation['path']

In [ ]:
sacceptation.sort_values('meanCoop')

In [ ]:
pd.set_option('display.max_colwidth', -1)
prev = None
for i, path in acceptation.sort_values('meanCoop')[-5:].iterrows():
    if prev is not None:
        print(', #', prev['id'])
    print("'" + path['path'] + "'", end="")
    prev = path
print(', #', prev['id'])


In [ ]:
pd.set_option("display.max_columns", -1 )

In [ ]:
def highlight_cols(s):
    color = '#CCFFCC'
    return f'background-color: {color} ; padding-left: 10px; border-left: 2px solid grey;'

def highlight_cols_red(s):
    color = '#EECCCC'
    return 'background-color: %s' % color

sacceptation_sorted = sacceptation.sort_values('meanCoop')
a = sacceptation_sorted.style.applymap(highlight_cols, subset=pd.IndexSlice[:, [column for column in sacceptation.columns if 'accept' in column]])
#print(sacceptation.sort_values('meanCoop').to_html())
a = a.applymap(highlight_cols_red, subset=pd.IndexSlice[:, [column for column in sacceptation.columns if 'reject' in column]])
out = a.render()
display(HTML('<textarea>' + out + '</textarea>'))
a

In [ ]:
sacceptation.style.applymap(highlight_cols, subset=pd.IndexSlice[:, ['_other_accept', '(3, 4]_accept']])
